In [217]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [218]:
paginas = ['']

In [219]:
for i in range (1, 400):
    a = str(i)
    paginas.append(a)

In [220]:
propriedades = []

for pg in paginas:
    url = 'https://www.vivareal.com.br/aluguel/sp/sao-jose-dos-campos/apartamento_residencial/#pagina={}'.format(pg)
    p = requests.get(url).content
    soup = BeautifulSoup(p)
    
    cards = soup.findAll("article", {"class":"js-property-card"})
    for card in cards:
        propriedade = {}
        propriedade["endereço"] = card.find("span", {"class": "property-card__address"}).text
        propriedade["área"] = card.find("li", {"class":"property-card__detail-area"}).text
        propriedade["quartos"] = card.find("li", {"class": "property-card__detail-room"}).text
        propriedade["banheiros"] = card.find("li", {"class":"property-card__detail-bathroom"}).text
        propriedade["vagas"] = card.find("li",{"class":"property-card__detail-garage"}).text
        propriedade["preço"] = card.find("div",{"class":"property-card__price"}).text
        propriedade["link"] = "https://www.vivareal.com.br"+card.find("div",attrs={"class":"property-card__carousel"}).a["href"]
        try:
            propriedade["condomínio"] = card.find("div",{"class":"property-card__price-details--condo"}).text
        except:
            propriedade["condomínio"] = None
        try:
            propriedade["outros"] = card.find("ul",{"class":"property-card__amenities"}).text
        except:
            propriedade["outros"] = None
        
        propriedades.append(propriedade)

In [221]:
df = pd.DataFrame(propriedades)

In [222]:
df.head()

,endereço,área,quartos,banheiros,vagas,preço,link,condomínio,outros
0,"Rua Professor Roberval Froes, 490 - Jardim Esp...",55 m²,2 Quartos,1 Banheiro,2 Vagas,R$ 1.990 /Mês,https://www.vivareal.com.br/imovel/apartamento...,Condomínio: R$ 350,Mobiliado Elevador Salão de festas Academia C...
1,"Estrada Municipal Martins Guimarães, 1201 - Vi...",47 m²,2 Quartos,1 Banheiro,1 Vaga,R$ 850 /Mês,https://www.vivareal.com.br/imovel/apartamento...,Condomínio: R$ 250,Área de serviço Churrasqueira Salão de festas...
2,"Avenida Engenheiro Francisco José Longo, 633 -...",120 m²,3 Quartos,4 Banheiros,1 Vaga,R$ 2.400 /Mês,https://www.vivareal.com.br/imovel/apartamento...,Condomínio: R$ 602,Mobiliado Elevador Área de serviço Salão de f...
3,"Avenida São João, 349 - Jardim Esplanada, São ...",125 m²,3 Quartos,4 Banheiros,3 Vagas,R$ 4.250 /Mês,https://www.vivareal.com.br/imovel/apartamento...,None,None
4,"Rua José Mattar, 174 - Jardim São Dimas, São J...",90 m²,3 Quartos,2 Banheiros,1 Vaga,R$ 1.300 /Mês,https://www.vivareal.com.br/imovel/apartamento...,Condomínio: R$ 570,Quadra poliesportiva


In [223]:
df['bairro']=df['endereço'].str.split('\s-').str[1]

In [224]:
df['bairro'] = df['bairro'].str.split(',').str[0]

In [225]:
df['rua'] = df['endereço'].str.split("\s-").str[0]

In [226]:
df['área'] = df['área'].str.split('\sm²').str[0]

In [227]:
df['quartos'] = df['quartos'].str.split('\sQuarto').str[0]

In [228]:
df['banheiros'] = df['banheiros'].str.split('\sBanheiro').str[0]

In [229]:
df['vagas'] = df['vagas'].str.split('\sVaga').str[0]

In [230]:
df['preço'] = df['preço'].str.split().str[1]

In [231]:
df['condomínio'] = df['condomínio'].str.split().str[2]

In [232]:
df.drop('endereço', inplace=True, axis=1)

In [233]:
df = df.rename(columns={'área': 'Área (m²)', 'quartos': 'Quartos', 'banheiros':'Banheiros','vagas':'Vagas','preço':'Preço (R$)', 'link':'Link', 'condomínio':'Condomínio (R$)', 'outros':'Outros','bairro':'Bairro','rua':'Rua'})

In [234]:
df.describe()

,Área (m²),Quartos,Banheiros,Vagas,Preço (R$),Link,Condomínio (R$),Outros,Bairro,Rua
count,14400,14400,14400,14400,14400,14400,14000,10336,14400,14400
unique,37,4,5,5,31,47,36,30,18,40
top,75,2,2,1,2.800,https://www.vivareal.com.br/imovel/apartamento...,600,Mobiliado Elevador Piscina Área de serviço Ch...,SP,Rua Doutor Tertuliano Delphim Júnior
freq,1488,8064,6400,7728,1200,400,912,1088,4928,800


In [235]:
df = df.drop_duplicates(subset=['Link'])

In [236]:
df.describe()

,Área (m²),Quartos,Banheiros,Vagas,Preço (R$),Link,Condomínio (R$),Outros,Bairro,Rua
count,47,47,47,47,47,47,46,36,47,47
unique,37,4,5,5,31,47,36,30,18,40
top,75,2,2,1,1.600,https://www.vivareal.com.br/imovel/apartamento...,400,Mobiliado Elevador Piscina Área de serviço Ch...,SP,"Jardim Aquarius, São José dos Campos"
freq,4,25,20,24,3,1,3,4,15,3
